## Demo of steady state temperature diffusion in an annulus

In [ ]:
import underworld as uw
import glucifer
import numpy as np

uw.matplotlib_inline()
import matplotlib.pyplot as plt
plt.ion()

In [ ]:
annulus = uw.mesh._FeMesh_Annulus(elementRes=(20,100), radialLengths=(3.0,6.), angularExtent=(0.0,360.0))
tField = uw.mesh.MeshVariable(annulus, nodeDofCount=1)

In [ ]:
fig = glucifer.Figure()
fig.append(glucifer.objects.Mesh(annulus))
fig.append(glucifer.objects.Surface(annulus, tField, onMesh=True))
fig.show()

In [ ]:
# fig.append(glucifer.objects.Mesh(annulus))
# fig.append(glucifer.objects.Surface(annulus, tField, onMesh=True))
fig.show()

In [ ]:
outer = annulus.specialSets["MaxI_VertexSet"]
inner = annulus.specialSets["MinI_VertexSet"]
tMin = 5.0
tMax = 10.0
tField.data[inner.data] = tMax
tField.data[outer.data] = tMin

In [ ]:
tBC = uw.conditions.DirichletCondition( variable=tField, indexSetsPerDof=(inner+outer))

In [ ]:
ssSLE = uw.systems.SteadyStateHeat(tField,fn_diffusivity=1.0, conditions=tBC)

In [ ]:
ssSolver = uw.systems.Solver(ssSLE)

In [ ]:
ssSolver.solve()

In [ ]:
fig.show()

In [ ]:
## Plot radial temperature and check against an analytic solution

nP = 100
# The 0.99 is required here as the mesh is not perfectly cylindrical
arrR = np.linspace(annulus.radialLengths[0],annulus.radialLengths[1]*0.99,nP)
                          
arrT = np.zeros(nP)
arrError = np.zeros(nP)
arrAnalytic = np.zeros(nP)

# Largest error at this angle (still not large)
theta = np.pi / 2.

for i in range(nP):
    pos = tuple(np.array( [ np.cos(theta),np.sin(theta)]) * (arrR[i]))
    t = tField.evaluate(pos)
    tAn = np.log( (arrR[i])/annulus.radialLengths[0]) / np.log(annulus.radialLengths[1]/annulus.radialLengths[0]) * (tMin-tMax)  + tMax
    
    arrT[i] = t
    arrAnalytic[i] = tAn
    arrError[i] = abs(t-tAn)

plotEvery = 9
plt.scatter(arrAnalytic[::plotEvery],arrR[::plotEvery],label="Analytic")

plt.plot(arrT,arrR,label="Numerical")
plt.ylabel("Radial Distance")
plt.xlabel("Temperature")
plt.legend()

In [ ]:
# Plot radial error, as a percentage of the max. temp difference
plt.plot(arrError / (tMax-tMin) * 100.,arrR)
plt.ylabel("Radial Distance")
plt.xlabel("Percentage Error")